In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pandas as pd
import pickle
import numpy as np

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [7]:
from batch_sv_system_utils import cosine_sim, compute_plda_score, get_embeds
from utils import get_id2idx

In [5]:
embed_dir = "embeddings/voxc2_fbank64_voxc2untied_embeds/"
sv_embeds = np.load(embed_dir + "/sv_embeds.npy")
keys = pickle.load(open(embed_dir + "/sv_keys.pkl", "rb"))
id2idx = get_id2idx(keys)

plda_embed_dir = "embeddings/voxc2_fbank64_voxc2untied_xvector/"
plda_sv_embeds = np.load(plda_embed_dir + "/sv_embeds.npy")
plda_model_dir = plda_embed_dir + "plda_train/"
plda_keys = pickle.load(open(plda_embed_dir + "/sv_keys.pkl", "rb"))
plda_id2idx = get_id2idx(plda_keys)

cohort_embeds = np.load("trials/dev940_eval311/dev_cohort_embeds.npy")

## Cosine  

In [18]:
a = get_embeds(keys[:5], sv_embeds, id2idx)
b = get_embeds(keys[3:10], sv_embeds, id2idx)

In [19]:
cosine_sim(a, b).mean(0)

array([0.79724103, 0.80605346, 0.7853865 , 0.79325885, 0.7850086 ,
       0.77598035, 0.7555752 ], dtype=float32)

In [21]:
# not exact cosine similarity of a.mean(0), because it does not have unit norm.
# however, it equals to the average of scores.
cosine_sim(a.mean(0), b)

array([0.797241  , 0.8060535 , 0.78538656, 0.79325885, 0.78500867,
       0.77598053, 0.7555753 ], dtype=float32)

## PLDA

In [35]:
a = get_embeds(plda_keys[:2], plda_sv_embeds, plda_id2idx, norm=False)
b = get_embeds(plda_keys[3:50], plda_sv_embeds, plda_id2idx, norm=False)

In [78]:
average_sort = np.argsort(compute_plda_score(a, b, plda_model_dir, mean=False).mean(0))

In [85]:
compute_plda_score(a, b, plda_model_dir, mean=True)

array([[12.91658 , 19.67509 , 19.25675 , 16.11789 , 39.27283 , 33.40192 ,
        32.1818  , 30.88167 , 21.53151 , 24.47119 , 34.96891 , 23.23224 ,
        27.14427 , 27.84225 , 13.10076 , 16.24495 ,  6.494434, 10.79073 ,
        24.23748 , 27.09806 , 34.80861 , 17.03445 , 17.71111 ,  8.220601,
        11.6761  , 18.70403 , 22.8391  , 18.69051 ,  8.714568, 27.09784 ,
         8.847136, 19.7783  , 23.19859 , 11.17869 , 18.75489 , 13.10855 ,
        24.95004 , 29.37314 , 25.55992 , 23.34107 ,  4.83798 , 17.37362 ,
        21.41072 , 27.01563 , 20.64428 , 13.75123 , 25.51117 ]])

In [88]:
np.argsort(compute_plda_score(a, b, plda_model_dir, mean=False).mean(0))

array([40, 16, 23, 28, 30, 17, 24, 35, 33,  0, 14, 45,  3, 15, 41, 21, 22,
       31,  1,  2, 25, 27, 34,  8, 44, 42, 11, 18, 26,  9, 39, 32, 36, 46,
       38, 43, 29, 19, 12, 13, 37,  7,  6,  5, 20, 10,  4])

In [87]:
np.argsort(compute_plda_score(a, b, plda_model_dir, mean=True))

array([[40, 16, 23, 28, 30, 17, 24, 35,  0, 33, 45, 14, 15, 41, 31,  3,
        21,  1, 25, 22,  2, 27,  8, 34, 44, 18,  9, 11, 42, 26, 39, 32,
        36, 46, 43, 29, 38, 19, 13, 12,  7, 37,  5,  6, 20, 10,  4]])

In [77]:
cent_sort = np.argsort(compute_plda_score(a, b, plda_model_dir, mean=True))[0]

In [79]:
average_sort

array([40, 16, 23, 28, 30, 17, 24, 35, 33,  0, 14, 45,  3, 15, 41, 21, 22,
       31,  1,  2, 25, 27, 34,  8, 44, 42, 11, 18, 26,  9, 39, 32, 36, 46,
       38, 43, 29, 19, 12, 13, 37,  7,  6,  5, 20, 10,  4])

In [80]:
cent_sort

array([40, 16, 23, 28, 30, 17, 24, 35,  0, 33, 45, 14, 15, 41, 31,  3, 21,
        1, 25, 22,  2, 27,  8, 34, 44, 18,  9, 11, 42, 26, 39, 32, 36, 46,
       43, 29, 38, 19, 13, 12,  7, 37,  5,  6, 20, 10,  4])

In [60]:
average_sort == cent_sort

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])